# The Battle of Neighborhood - Week 1

## Introduction

### Business Case:-

New york city is one of the most popular city in the world, with population of nearly 9 million and it is the most densely populated major city in the US.

An Investor was looking to open a restaurant at Manhattan, New York. However, they are unsure on which location would be strategically beneficial to their future business.

To open a restaurant, other than the food needed to be unique at its own, we will definitely need to consider a strategic location for the restaurant to be accessable to all locals.

Therefore, we are require to analysed data to provide a clearer image on which area will be the suitable for a restaurant to be located and will be profitable enough to help our client to expand its buisness in future.


### Background discussion:-

By utilising Foursquare API, we are able able to collect competitors location around the area which allow us to have a better insight on which Food is more appealing towards the customer.

We will also need to find out the population at the neighbourhood in order to help us determine which strategic area is the best location to invest on.

### To Solve:-

#### To find the most populated area in Manhattan

First we need to find out which Neighborhood in Manhattan is the most populated
by importing a CSV file from NYC data

In [1]:
# The code was removed by Watson Studio for sharing.

,Borough,Year,FIPS County Code,NTA Code,NTA Name,Population
0,Manhattan,2000,61,MN01,Marble Hill-Inwood,49087
1,Manhattan,2000,61,MN03,Central Harlem North-Polo Grounds,70016
2,Manhattan,2000,61,MN04,Hamilton Heights,50555
3,Manhattan,2000,61,MN06,Manhattanville,24772
4,Manhattan,2000,61,MN09,Morningside Heights,54208


In [3]:
df_data_1.sort_values('Population', ascending=False).head()

,Borough,Year,FIPS County Code,NTA Code,NTA Name,Population
6,Manhattan,2000,61,MN12,Upper West Side,136954
35,Manhattan,2010,61,MN12,Upper West Side,132378
25,Manhattan,2000,61,MN36,Washington Heights South,93671
54,Manhattan,2010,61,MN36,Washington Heights South,84438
49,Manhattan,2010,61,MN31,Lenox Hill-Roosevelt Island,80771


From the data above, shows that Upper West Side area is the most populated area in Manhattan.

### To visuallize Neightborhood in New York City

In [4]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import json # library to handle JSON files

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [5]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [6]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [7]:
neighborhoods_data = newyork_data['features']

In [8]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [9]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [10]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [12]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [13]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### To Visualize Manhattan Neighborhood

In [14]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [15]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [16]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

#### To Visualize Manhattan Neighborhood

In [17]:
CLIENT_ID = '3BYCUI5GNTJLYZITXI1AVECXPZRWK2NTAUQO2C1QJRGNI3UG' # your Foursquare ID
CLIENT_SECRET = 'BVSBU2CECUFN4C4QEQALK1HPTBNUDZ44DPODMHA5HGNACPID' # your Foursquare Secret
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3BYCUI5GNTJLYZITXI1AVECXPZRWK2NTAUQO2C1QJRGNI3UG
CLIENT_SECRET:BVSBU2CECUFN4C4QEQALK1HPTBNUDZ44DPODMHA5HGNACPID


In [18]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [19]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [20]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=3BYCUI5GNTJLYZITXI1AVECXPZRWK2NTAUQO2C1QJRGNI3UG&client_secret=BVSBU2CECUFN4C4QEQALK1HPTBNUDZ44DPODMHA5HGNACPID&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e92f30ebe61c9001be674d6'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 25,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label'

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Dunkin',Donut Shop,40.877136,-73.906666
4,Starbucks,Coffee Shop,40.877531,-73.905582


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

25 venues were returned by Foursquare.


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [27]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3176, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [28]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,70,70,70,70,70,70
Carnegie Hill,90,90,90,90,90,90
Central Harlem,41,41,41,41,41,41
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,41,41,41,41,41,41
East Village,100,100,100,100,100,100


In [29]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 329 uniques categories.


In [30]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
manhattan_onehot.shape

(3176, 330)

In [32]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.000000,0.00,0.00,0.00000,0.014286,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.042857,0.000000,0.014286,0.000000
1,Carnegie Hill,0.000000,0.00,0.00,0.00000,0.011111,0.00,0.000000,0.000000,0.000000,...,0.00,0.011111,0.000000,0.00,0.000000,0.011111,0.033333,0.000000,0.011111,0.033333
2,Central Harlem,0.000000,0.00,0.00,0.04878,0.048780,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Chelsea,0.000000,0.00,0.00,0.00000,0.040000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.010000,0.010000,0.000000,0.000000,0.010000
4,Chinatown,0.000000,0.00,0.00,0.00000,0.040000,0.00,0.000000,0.000000,0.000000,...,0.00,0.030000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000
5,Civic Center,0.000000,0.00,0.00,0.00000,0.030000,0.01,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.010000,0.020000,0.010000,0.000000,0.030000
6,Clinton,0.000000,0.00,0.00,0.00000,0.030000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.010000,0.030000,0.000000,0.000000,0.000000
7,East Harlem,0.000000,0.00,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,East Village,0.000000,0.00,0.00,0.00000,0.020000,0.00,0.000000,0.010000,0.010000,...,0.00,0.020000,0.000000,0.00,0.000000,0.040000,0.010000,0.000000,0.000000,0.000000
9,Financial District,0.000000,0.00,0.00,0.00000,0.030000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.010000,0.010000


In [33]:
manhattan_grouped.shape

(40, 330)

In [34]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.09
1          Hotel  0.07
2            Gym  0.06
3  Memorial Site  0.04
4    Coffee Shop  0.04


----Carnegie Hill----
                  venue  freq
0           Coffee Shop  0.08
1                  Café  0.06
2        Cosmetics Shop  0.03
3             Bookstore  0.03
4  Gym / Fitness Center  0.03


----Central Harlem----
                 venue  freq
0    French Restaurant  0.05
1   African Restaurant  0.05
2  American Restaurant  0.05
3                  Bar  0.05
4   Chinese Restaurant  0.05


----Chelsea----
                 venue  freq
0          Coffee Shop  0.09
1          Art Gallery  0.05
2   Italian Restaurant  0.04
3  American Restaurant  0.04
4       Ice Cream Shop  0.03


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.09
1         Cocktail Bar  0.05
2  American Restaurant  0.04
3               Bakery  0.04
4         Dessert Shop  0.03


----Civic Center----
               venue  fr

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Boat or Ferry,Coffee Shop,Memorial Site,Wine Shop,Gourmet Shop,Mexican Restaurant,Shopping Mall
1,Carnegie Hill,Coffee Shop,Café,Yoga Studio,Gym,Pizza Place,Bookstore,Bakery,Japanese Restaurant,Cosmetics Shop,Italian Restaurant
2,Central Harlem,Seafood Restaurant,Bar,African Restaurant,American Restaurant,Chinese Restaurant,French Restaurant,Dessert Shop,Library,Gym / Fitness Center,Beer Bar
3,Chelsea,Coffee Shop,Art Gallery,American Restaurant,Italian Restaurant,Ice Cream Shop,Bakery,Gym / Fitness Center,Speakeasy,Hotel,Nightclub
4,Chinatown,Chinese Restaurant,Cocktail Bar,Bakery,American Restaurant,Spa,Dessert Shop,Salon / Barbershop,Vietnamese Restaurant,Hotpot Restaurant,Optical Shop


In [37]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 1, 4, 0, 1, 4, 1, 1], dtype=int32)

In [38]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,3,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Deli / Bodega,Supplement Shop,Steakhouse,Shopping Mall,Seafood Restaurant,Pizza Place
1,Manhattan,Chinatown,40.715618,-73.994279,4,Chinese Restaurant,Cocktail Bar,Bakery,American Restaurant,Spa,Dessert Shop,Salon / Barbershop,Vietnamese Restaurant,Hotpot Restaurant,Optical Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Café,Bakery,Grocery Store,Deli / Bodega,Chinese Restaurant,Mobile Phone Shop,Mexican Restaurant,Supplement Shop,Coffee Shop,Latin American Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,4,Mexican Restaurant,Café,Lounge,Pizza Place,Restaurant,Chinese Restaurant,Park,Bakery,American Restaurant,Frozen Yogurt Shop
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Sushi Restaurant,Cocktail Bar,Sandwich Place,Bakery,Yoga Studio


In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [43]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Italian Restaurant,Coffee Shop,Bakery,Gym / Fitness Center,Exhibit,Yoga Studio,Wine Shop,Juice Bar,French Restaurant,Spa
9,Yorkville,Italian Restaurant,Coffee Shop,Gym,Bar,Deli / Bodega,Pizza Place,Sushi Restaurant,Wine Shop,Diner,Japanese Restaurant
10,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Cocktail Bar,Café,Burger Joint,Gym,Gym / Fitness Center,Playground
12,Upper West Side,Italian Restaurant,Bakery,Bar,Coffee Shop,Thai Restaurant,Shoe Store,Seafood Restaurant,Café,Bagel Shop,Ice Cream Shop
14,Clinton,Italian Restaurant,Gym / Fitness Center,Theater,Coffee Shop,Spa,Thai Restaurant,Cocktail Bar,Sandwich Place,Gym,Hotel
15,Midtown,Coffee Shop,Hotel,Theater,Sandwich Place,Sporting Goods Shop,Steakhouse,Gym,Clothing Store,Café,Bakery
16,Murray Hill,Sandwich Place,Hotel,Bar,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Pizza Place,Burger Joint,Gym / Fitness Center,Mediterranean Restaurant
17,Chelsea,Coffee Shop,Art Gallery,American Restaurant,Italian Restaurant,Ice Cream Shop,Bakery,Gym / Fitness Center,Speakeasy,Hotel,Nightclub
18,Greenwich Village,Italian Restaurant,Café,French Restaurant,Sushi Restaurant,Gym,Dessert Shop,Coffee Shop,Chinese Restaurant,Sandwich Place,Pilates Studio
19,East Village,Bar,Pizza Place,Ice Cream Shop,Mexican Restaurant,Wine Bar,Coffee Shop,Cocktail Bar,Speakeasy,Ramen Restaurant,Korean Restaurant


From the result above, the 1st Most Common Venue at Upper East Side is Italian Restaurant

Therefore, It will be a wiser choice to open an Italian restaurant at Upper East Side area.

### To Find Nearby Italian Restaurant (competition) around Upper East Side area

For the convenience of the customer to visit an Restaurant, the location of the restaurant needed to be closer to the Rail way or bus station.

In this case we have chosen the 86th street subway station so that it will also be accessable for anyone who doesnt have a Car to travel to the restaurant.

In [58]:
CLIENT_ID = '3BYCUI5GNTJLYZITXI1AVECXPZRWK2NTAUQO2C1QJRGNI3UG' # your Foursquare ID
CLIENT_SECRET = 'BVSBU2CECUFN4C4QEQALK1HPTBNUDZ44DPODMHA5HGNACPID' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3BYCUI5GNTJLYZITXI1AVECXPZRWK2NTAUQO2C1QJRGNI3UG
CLIENT_SECRET:BVSBU2CECUFN4C4QEQALK1HPTBNUDZ44DPODMHA5HGNACPID


In [82]:
address = '86th Street & Second Avenue, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.9459995 -72.972936


In [83]:
search_query = 'Italian'
radius = 8000
print(search_query + ' .... OK!')

Italian .... OK!


In [84]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=3BYCUI5GNTJLYZITXI1AVECXPZRWK2NTAUQO2C1QJRGNI3UG&client_secret=BVSBU2CECUFN4C4QEQALK1HPTBNUDZ44DPODMHA5HGNACPID&ll=40.9459995,-72.972936&v=20180605&query=Italian&radius=8000&limit=30'

In [85]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e92fdbbe826ac001be57ad1'},
 'response': {'venues': [{'id': '4ba51341f964a52084d938e3',
    'name': "Ralph's Famous Italian Ices",
    'location': {'address': '745 Route 25A',
     'lat': 40.94374006840455,
     'lng': -72.97418727277145,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.94374006840455,
       'lng': -72.97418727277145},
      {'label': '?', 'lat': 40.943784, 'lng': -72.974149}],
     'distance': 272,
     'postalCode': '11764',
     'cc': 'US',
     'city': 'Miller Place',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['745 Route 25A',
      'Miller Place, NY 11764',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1c9941735',
      'name': 'Ice Cream Shop',
      'pluralName': 'Ice Cream Shops',
      'shortName': 'Ice Cream',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_',
       'suffix': '.png'},
      'primary': True}],
    'referra

In [86]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",False,4ba51341f964a52084d938e3,745 Route 25A,US,Miller Place,United States,272,"[745 Route 25A, Miller Place, NY 11764, United...","[{'label': 'display', 'lat': 40.94374006840455...",40.943740,-72.974187,11764,NY,Ralph's Famous Italian Ices,v-1586691683
1,"[{'id': '4bf58dd8d48988d1f5941735', 'name': 'G...",False,4dfe432cb0fb709b61e330af,39 Broadway,US,Rocky Point,United States,4164,"[39 Broadway, Rocky Point, NY 11778, United St...","[{'label': 'display', 'lat': 40.948493, 'lng':...",40.948493,-72.923516,11778,NY,Del Fiore Italian Market,v-1586691683
2,[],False,4defd2c3887754a6af9cb85b,NaN,US,Coram,United States,7219,"[Coram, NY 11727, United States]","[{'label': 'display', 'lat': 40.8895052149009,...",40.889505,-73.015086,11727,NY,Italian Fishermen,v-1586691683
3,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",False,4f954303e4b06ce859922547,2229 Route 112,US,Coram,United States,8872,"[2229 Route 112, Coram, NY 11727, United States]","[{'label': 'display', 'lat': 40.86971448292171...",40.869714,-73.003506,11727,NY,Ralph's Famous Italian Ices,v-1586691683
4,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",False,4bba30277421a593cf80c340,112 Main St,US,Port Jefferson,United States,8065,"[112 Main St, Port Jefferson, NY 11777, United...","[{'label': 'display', 'lat': 40.94621538943398...",40.946215,-73.068864,11777,NY,Ralph's Famous Italian Ices,v-1586691683


In [87]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Ralph's Famous Italian Ices,Ice Cream Shop,745 Route 25A,US,Miller Place,United States,272,"[745 Route 25A, Miller Place, NY 11764, United...","[{'label': 'display', 'lat': 40.94374006840455...",40.943740,-72.974187,11764,NY,4ba51341f964a52084d938e3
1,Del Fiore Italian Market,Gourmet Shop,39 Broadway,US,Rocky Point,United States,4164,"[39 Broadway, Rocky Point, NY 11778, United St...","[{'label': 'display', 'lat': 40.948493, 'lng':...",40.948493,-72.923516,11778,NY,4dfe432cb0fb709b61e330af
2,Italian Fishermen,None,NaN,US,Coram,United States,7219,"[Coram, NY 11727, United States]","[{'label': 'display', 'lat': 40.8895052149009,...",40.889505,-73.015086,11727,NY,4defd2c3887754a6af9cb85b
3,Ralph's Famous Italian Ices,Ice Cream Shop,2229 Route 112,US,Coram,United States,8872,"[2229 Route 112, Coram, NY 11727, United States]","[{'label': 'display', 'lat': 40.86971448292171...",40.869714,-73.003506,11727,NY,4f954303e4b06ce859922547
4,Ralph's Famous Italian Ices,Ice Cream Shop,112 Main St,US,Port Jefferson,United States,8065,"[112 Main St, Port Jefferson, NY 11777, United...","[{'label': 'display', 'lat': 40.94621538943398...",40.946215,-73.068864,11777,NY,4bba30277421a593cf80c340
5,Napoli Italian Market,Italian Restaurant,562 Jefferson Plz,US,Port Jefferson Station,United States,6873,"[562 Jefferson Plz, Port Jefferson Station, NY...","[{'label': 'display', 'lat': 40.92946, 'lng': ...",40.929460,-73.051688,11776,NY,4f323b7f19836c91c7c3384f
6,A and S Italian Pork Store,Food,4747 Nesconset Hwy,US,Port Jefferson Station,United States,7425,"[4747 Nesconset Hwy, Port Jefferson Station, N...","[{'label': 'display', 'lat': 40.918201, 'lng':...",40.918201,-73.053200,11776,NY,4f32806e19836c91c7de01a8
7,Lombardi's Italian Restaurant,Italian Restaurant,185 Middle Country Rd,US,Middle Island,United States,7675,"[185 Middle Country Rd, Middle Island, NY 1195...","[{'label': 'display', 'lat': 40.877077, 'lng':...",40.877077,-72.970405,11953,NY,4f32840419836c91c7df5e69
8,Ciro's Barber Shop and Salon,Salon / Barbershop,230 Middle Country Rd,US,Selden,United States,10420,"[230 Middle Country Rd, Selden, NY 11784, Unit...","[{'label': 'display', 'lat': 40.86610321708888...",40.866103,-73.037469,11784,NY,4efdfbdb6c25c5ce7f7f5ddc


In [88]:
dataframe_filtered.name

0      Ralph's Famous Italian Ices
1         Del Fiore Italian Market
2                Italian Fishermen
3      Ralph's Famous Italian Ices
4      Ralph's Famous Italian Ices
5            Napoli Italian Market
6       A and S Italian Pork Store
7    Lombardi's Italian Restaurant
8     Ciro's Barber Shop and Salon
Name: name, dtype: object

In [89]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the InterContinental Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='InterContinental Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

##### The result above shows that there are only 1 or 2 Italian restaurant around Upper East Side neighborhood.

#### Therefore, it will be a wise choice to open an Italian restaurant at preferbly walking distance from 86th street subway station. so that, everyone can enjoy a tasty meal from the restaurant.